# Notebook for Lab 1 - Symmetric Encryption

## General goal
In this lab you’ll get familiarized with programming (*using*) symmetric encryption algorithms using a modern library. The lab makes use of Python language but it could also be done with Java or C-like languages. 

## Specific goals
- Make use of different data representation formats (byte/ascii, string, hexadecimal, b64). Students should be able to recognize each of the formats and translate from one format to another.
- Make use of the pyca/cryptography library to encrypt and decrypt simple messages (strings) with block ciphers (AES, Camellia) operated under different modes (ECB, CBC and CTR) and considering padding issues.
- Make use of a robust pseudo random number generator function/algorithm to generate keys, initialization vectors and nonces.
- Understand how modifications in the ciphertext affect the decryption result when different operation modes are used.
- Make use of the pyca/cryptography library to encrypt and decrypt simple messages with stream ciphers (ChaCha20).

## Modules
Cryptography library includes both high level recipes and low-level interfaces to common cryptographic algorithms such as symmetric ciphers, message digests, and key derivation functions. For more details you can consult the following link: https://pypi.org/project/cryptography. For the moment, we are going to directly use the primitives, forgetting about Fernet constructions. You’ll may find useful to have these links as reference:
- Links to the documentation of the specific sections at stake in this lab:
- Symmetric encryption: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/
- Symmetric padding: https://cryptography.io/en/latest/hazmat/primitives/padding/
- Link to info about os library: https://docs.python.org/3/library/os.html
- Link to info about base64 library: https://docs.python.org/3/library/base64.html
- Link to your very first encryption/decryption code snippet example: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher

## Lab assignment and assessment
You are given one Python script/notebook, that contains the headers of all the functions you should develop at the end of this lab. Next, you’ll be guided through several tasks you’ll have to complete in order to do so.

Lab 1 assessment will be performed with a test/quiz/exam at the end of the course (along the other lab assessments).

# 0. Modules installation and imports

In [4]:
!pip install cryptography

In [5]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

from cryptography.hazmat.primitives import padding

import base64

import os 

import secrets

# 1. Symmetric Block Ciphers

Next, you’ll learn how to encrypt a simple text with symmetric block ciphers. First, you’ll use AES cipher operated under several confidentiality operation modes, and then you’ll be prompted to use other symmetric ciphers (CAMELLIA).
Read AND USE AS REFERENCE the information provided in the documentation web page of the pyca/cryptography library for symmetric encryption: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/ 

## 1.1 BYTE-LIKE DATA

### Task 1.1.1 From byte-like to hex and b64

First, let’s see a primer in dealing with binary data in Python, as it is a very frequent data format when using cryptography. The code lines for defining the key, the initialization vector and the text that you’ll use in your first encryption are given next:

In [2]:
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
iv = b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
message = b'a secret message'

Notice the preceding “b” in the previous data in single quotes. It is used to transform the provided (“text”-like) data into binary format (following ASCII codification https://en.wikipedia.org/wiki/ASCII). That is, the code will treat these data as binary. Each byte can be specified by:
- the ASCII character corresponding to that byte if it is printable, 
- the hexadecimal representation of the byte, preceding the 2-hex digits by “\x”.

Some examples: 
- b’MN’ corresponds to binary data 4d4e <sub>(16</sub> = 0100 1101 0100 1110 <sub>(2</sub> 
- b’\xa1\xf7’ corresponds to binary data a1f7 <sub>(16</sub> = 1010 0001 1111 0111 <sub>(2</sub>

You can get the hexadecimal representation of binary data using the built-in Python function <code>hex()</code> as follows (note: <code>hex()</code> belongs to the set of Python built-in functions (https://docs.python.org/3/library/functions.html):

In [5]:
binarydata = b'MN\\\xde\xa1' 
hex_binarydata = binarydata.hex() 
print("hex_binarydata =", hex_binarydata)

hex_binarydata = 4d4e5cdea1


You can get back your bytes from their hex representation as follows using <code>bytes.fromhex()</code> (https://docs.python.org/3/library/stdtypes.html?highlight=fromhex#bytes.fromhex):

In [6]:
retrieved_binarydata = bytes.fromhex(hex_binarydata) 
print("retrieved_binarydata =", retrieved_binarydata)

retrieved_binarydata = b'MN\\\xde\xa1'


Other codification that can be used to deal in a print-friendly manner with binary data is B64 (https://en.wikipedia.org/wiki/Base64). 

An option is to use the Python module base64 (https://docs.python.org/3/library/base64.html). In order to use it you have to import the module in your code (you should have done it in section 0) and use <code>base64.b64encode()</code> as follows:

In [7]:
# import base64 

b64_binarydata = base64.b64encode(binarydata)
print("b64_binarydata =",b64_binarydata)

b64_binarydata = b'TU5c3qE='


As before, you can get your bytes back from B64 as follows using <code>base64.b64decode()</code>:

In [8]:
retrieved_binarydata = base64.b64decode(b64_binarydata)
print("retrieved_binarydata =", retrieved_binarydata)

retrieved_binarydata = b'MN\\\xde\xa1'


#### TASK
<mark>Represent the given key, IV and message text in hexadecimal and B64 and check that you understand how binary data is provided and represented. Undo the hex and B64 representations by retrieving back the bytes and print the results.</mark>

In [3]:
# Importing necessary libraries
import base64

# Key hex & b64
key_hex = key.hex()
key_b64 = base64.b64encode(key)
print(f"Hexadecimal represenation of the key is {key_hex}")
print(f"Base64 representation of the key is {key_b64}")

# IV hex & 64
iv_hex = iv.hex()
iv_b64 = base64.b64encode(iv)
print(f"Hexadecimal represenation of IV is {iv_hex}")
print(f"Base64 representation of IV is {iv_b64}")

# Key original
decrypted_key_hex = bytes.fromhex(key_hex)
decrypted_key_b64 = base64.b64decode(key_b64)
print(f"Original key byte representation from hex representation is {decrypted_key_hex}")
print(f"Original key byte representation from b64 representation is {decrypted_key_b64}")

# IV original
decrypted_iv_hex = bytes.fromhex(iv_hex)
decrypted_iv_b64 = base64.b64decode(iv_b64)
print(f"Original IV byte representation from hex representation is {decrypted_iv_hex}")
print(f"Original IV byte representation from b64 representation is {decrypted_iv_b64}")

Hexadecimal represenation of the key is 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
Base64 representation of the key is b'TU5c3qH3nM8QQageAA5lceg2HBIYphD7uV4KLDCia2o='
Hexadecimal represenation of IV is 7fb357841a394b4e7eaf73fc9fb6ccb8
Base64 representation of IV is b'f7NXhBo5S05+r3P8n7bMuA=='
Original key byte representation from hex representation is b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
Original key byte representation from b64 representation is b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
Original IV byte representation from hex representation is b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
Original IV byte representation from b64 representation is b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'


## 1.2 AES256 - CBC MODE

### Task 1.2.1 Encrypt/decrypt with AES 256 CBC

Now, let’s encrypt a simple text (message) using the AES cipher with a 256-bit key and operating the cipher under CBC mode.

Read the information and take the code provided as example in the following web page section:

https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher 

Notice that Cipher is a general construction that can be instantiated with a specific algorithm, mode and backend (note: We will rely on the default backend but if you want to know more about backends, read this web page: https://cryptography.io/en/latest/hazmat/backends/). 

In this case we’ll use the classes <code>algorithms.AES(..)</code> and <code>modes.CBC(..)</code> for the first two parameters.

Notice also that in the example in the web page
https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.Cipher
the key and iv data are randomly obtained.

#### TASK

<mark>Copy and change the code of the example (given at the web page of the library) so you use the key, iv and message given in Section 1.1. Leave the rest of the code unchanged. Additionally, insert calls to the <code>print()</code> function so the message, key, iv and the results of the encryption and decryption processes are printed. </mark>

In [9]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64

cipher = Cipher(algorithms.AES(key), modes.CBC(iv))

print(f"message = {message}")
print(f"key = {key}")

## encryption code
encryptor = cipher.encryptor()
ct = encryptor.update(message) + encryptor.finalize()
print("ciphertext =", ct)
print("hex_ciphertext = ", ct.hex())
print("b64_ciphertext = ", base64.b64encode(ct))

## decryption code
decryptor = cipher.decryptor()
pt = decryptor.update(ct) + decryptor.finalize()
print("plaintext =", pt)
print("hex_plaintext = ", pt.hex())
print("b64_plaintext = ", base64.b64encode(pt))

message = b'a secret message'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
hex_ciphertext =  3ba6e5853b5ec226ec077c03cc0dae4e
b64_ciphertext =  b'O6blhTtewibsB3wDzA2uTg=='
plaintext = b'a secret message'
hex_plaintext =  6120736563726574206d657373616765
b64_plaintext =  b'YSBzZWNyZXQgbWVzc2FnZQ=='


#### CHECK
Check your results with the following values:
<code>
message = b'a secret message'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
hex_ciphertext = 3ba6e5853b5ec226ec077c03cc0dae4e
b64_ciphertext = b'O6blhTtewibsB3wDzA2uTg=='
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
b64_plaintext = b'YSBzZWNyZXQgbWVzc2FnZQ=='
</code>

#### QUESTION

<mark>Which length in bytes has the input data to encrypt in Task 1.2.2? </mark>

<mark>And the output ciphertext? </mark>

Notice that you may use the built-in function <code>len(..)</code> to get the length of an object (https://docs.python.org/3/library/functions.html#len).

In [8]:
# YOUR CODE HERE
print(f"Length of the input data: {len(message)}")

print(f"Length of the ciphertext: {len(ct)}")

Length of the input data: 16
Length of the ciphertext: 16


YOUR ANSWER HERE

The input text has a length of 16 bytes, along with the ciphertext.

### Task 1.2.2 Adding padding to AES 256 CBC

Now let’s play with the length of the message to encrypt! In the previous exercise you encrypted a message that has an exact length of 16 characters, that is, the AES block size. Try to encrypt a longer message whose length in characters is not multiple of 16. You’ll get an error for sure! It is because if you need to encrypt a message with length not multiple of the cipher block size and you are using CBC operation mode, you need to add padding to your input so its length satisfies that condition.

Read the information provided in the documentation web page of the pyca/cryptography library on how the library deals with symmetric padding:
https://cryptography.io/en/latest/hazmat/primitives/padding/

Study the code in the example provided to illustrate the PKCS7 padding here:
https://cryptography.io/en/latest/hazmat/primitives/padding/#cryptography.hazmat.primitives.padding.PKCS7

Notice that you have to import the padding module from the pyca/cryptography library (if you have not done it in Section 0). Notice also that the correct order to combine padding-encryption and decryption-unpadding is the following one:
1. Pad the data to encrypt to obtain the padded data
2. Encrypt the padded data to obtain the encrypted text
3. --- (simulation of data exchange) ----
4. Decrypt the encrypted text to obtain the padded plaintext
5. Unpad the padded plaintext to obtain the original plaintext

#### TASK 
<mark>After studying it, incorporate PKCS7 padding in your previous code. </mark>

In [10]:
import os
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64

cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
padder = padding.PKCS7(128).padder()

print(f"message = {message}")

padded_message = padder.update(message) + padder.finalize()

print(f"padded_message = {padded_message}   length = {len(padded_message)}")

## encryption code
encryptor = cipher.encryptor()
ct = encryptor.update(padded_message) + encryptor.finalize()
print("ciphertext =", ct)
print("hex_ciphertext = ", ct.hex())
print("b64_ciphertext = ", base64.b64encode(ct))

## decryption code
decryptor = cipher.decryptor()
pt = decryptor.update(ct) + decryptor.finalize()
print("padded_plaintext =", pt)
print("hex_padded_plaintext = ", pt.hex())

unpadder = padding.PKCS7(128).unpadder()
unpadded_pt = unpadder.update(pt) + unpadder.finalize()
print("plaintext = ", unpadded_pt)
print("hex_plaintext = ", unpadded_pt.hex())

message = b'a secret message'
padded_message = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'   length = 32
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeNq\xb6\xb8\xb5\x06\xeeJ\xd9\xfb\x004\xb5\xac\x9e}\xf0'
hex_ciphertext =  3ba6e5853b5ec226ec077c03cc0dae4e71b6b8b506ee4ad9fb0034b5ac9e7df0
b64_ciphertext =  b'O6blhTtewibsB3wDzA2uTnG2uLUG7krZ+wA0tayeffA='
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext =  6120736563726574206d65737361676510101010101010101010101010101010
plaintext =  b'a secret message'
hex_plaintext =  6120736563726574206d657373616765


#### CHECK
Your previous AES256-CBC code, updated to include PKCS7 padding, with the data provided in Section 1.1 (key, iv and message), should provide the result shown next:
<code>
message = b'a secret message'
padded_message = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10' length =  32
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeNq\xb6\xb8\xb5\x06\xeeJ\xd9\xfb\x004\xb5\xac\x9e}\xf0'
hex_ciphertext = 3ba6e5853b5ec226ec077c03cc0dae4e71b6b8b506ee4ad9fb0034b5ac9e7df0
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext = 6120736563726574206d65737361676510101010101010101010101010101010
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
</code>

#### QUESTION

<mark>Which length in bytes has the input data to encrypt in Task 1.2.2? </mark>

<mark>Which length in bytes has the output ciphertext in Task 1.2.2? </mark>

<mark>How many bytes have been added as PKCS7 padding? </mark>

<mark>Which value has been used in the bytes added as padding? </mark>

In [13]:
# YOUR CODE HERE
print(f"Length of input data: {len(padded_message)}")
print(f"Length of output ciphertext: {len(ct)}")
print(f"Number of bytes added as PKC57 padding: {len(padded_message) - len(message)}")

Length of input data: 32
Length of output ciphertext: 32
Number of bytes added as PKC57 padding: 16


The length of the input data in bytes is 32
The length of the output ciphertext in bytes is 32
The number of bytes that were added in padding was 16
The value used as padding is '\x10'

### Task 1.2.3 Checking AES 256 CBC PKCS7

#### TASK

<mark>Check that your code works correctly by encrypting <code>other_message</code>, a message with 25 characters (length not multiple of 16) and provided next, and the same <code>key</code> and <code>iv</code> already used. </mark>

In [15]:
import os
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64

cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
padder = padding.PKCS7(128).padder()

other_message = b'This message is important'
print(f"message = {other_message}")

padded_message = padder.update(other_message) + padder.finalize()

print(f"padded_message = {padded_message}   length = {len(padded_message)}")

## encryption code
encryptor = cipher.encryptor()
ct = encryptor.update(padded_message) + encryptor.finalize()
print("ciphertext =", ct)
print("hex_ciphertext = ", ct.hex())
print("b64_ciphertext = ", base64.b64encode(ct))

## decryption code
decryptor = cipher.decryptor()
pt = decryptor.update(ct) + decryptor.finalize()
print("padded_plaintext =", pt)
print("hex_padded_plaintext = ", pt.hex())

unpadder = padding.PKCS7(128).unpadder()
unpadded_pt = unpadder.update(pt) + unpadder.finalize()
print("plaintext = ", unpadded_pt)
print("hex_plaintext = ", unpadded_pt.hex())

message = b'This message is important'
padded_message = b'This message is important\x07\x07\x07\x07\x07\x07\x07'   length = 32
ciphertext = b'\x89\x00g\x1aP4\xf7W\x16\xc1\xffi\xbb\xc8\x9f\xcf\xd3\x84D\x1d\nL\xcbvb\xf1^sn\xfa\x06Z'
hex_ciphertext =  8900671a5034f75716c1ff69bbc89fcfd384441d0a4ccb7662f15e736efa065a
b64_ciphertext =  b'iQBnGlA091cWwf9pu8ifz9OERB0KTMt2YvFec276Blo='
padded_plaintext = b'This message is important\x07\x07\x07\x07\x07\x07\x07'
hex_padded_plaintext =  54686973206d65737361676520697320696d706f7274616e7407070707070707
plaintext =  b'This message is important'
hex_plaintext =  54686973206d65737361676520697320696d706f7274616e74


#### CHECK
You should get the following results:
<code>
other_message = b'This message is important'
padded_other_message = b'This message is important\x07\x07\x07\x07\x07\x07\x07'
length =  32
ciphertext = b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeNq\xb6\xb8\xb5\x06\xeeJ\xd9\xfb\x004\xb5\xac\x9e}\xf0'
hex_ciphertext = 3ba6e5853b5ec226ec077c03cc0dae4e71b6b8b506ee4ad9fb0034b5ac9e7df0
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext = 6120736563726574206d65737361676510101010101010101010101010101010
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
</code>

#### QUESTION

<mark>Which length in bytes has the input data to encrypt in Task 1.2.3?</mark>

<mark>Which length in bytes has the output ciphertext in Task 1.2.3?</mark>

<mark>How many bytes have been added as PKCS7 padding?</mark> 

<mark>Which value has been used in the bytes added as padding?</mark>

In [17]:
# YOUR CODE HERE
print(f"Length of input data: {len(padded_message)}")
print(f"Length of output ciphertext: {len(ct)}")
print(f"Number of bytes added as PKC57 padding: {len(padded_message) - len(other_message)}")

Length of input data: 32
Length of output ciphertext: 32
Number of bytes added as PKC57 padding: 7


The length of the input data is 32 bytes
The length of the output ciphertext is 32 bytes
The number of bytes added as part of padding is 16 bytes
The value used for padding was \x07

### Task 1.2.4 Wrapping AES 256 CBC PKCS7 encryption/decryption in functions

#### TASK

<mark>Let’s wrap your AES256-CBC-PKCS7 code in two functions, one for encryption and another one for decryption. Next you have the headers that you should use for these two functions.</mark>

In [28]:
def aes256_cbc_pkcs7_encrypt(data_to_encrypt, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    padder = padding.PKCS7(128).padder()

    padded_message = padder.update(data_to_encrypt) + padder.finalize()

    encryptor = cipher.encryptor()
    ct = encryptor.update(padded_message) + encryptor.finalize()
    
    return ct

def aes256_cbc_pkcs7_decrypt(encrypted_data, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    unpadder = padding.PKCS7(128).unpadder()
    decryptor = cipher.decryptor()
    padded_pt = decryptor.update(encrypted_data) + decryptor.finalize()
    unpadded_pt = unpadder.update(padded_pt) + unpadder.finalize()
    
    return unpadded_pt

#### CHECK

Test your functions with the data and results of Task 1.2.3

### Task 1.2.5 Checking AES 256 CBC PKCS7 against CyberChef

#### TASK

<mark> Test again your AES256-CBC-PKCS7 function but this time check your results using the web page tool CyberChef: https://gchq.github.io/CyberChef/ . You have to select, in the left sidebar, 'AES Encrypt' from 'Encryption / Encoding' and CBC mode.  </mark>

## 1.3 AES256 - ECB MODE

Let’s now use the ECB operation mode. It is the simplest of all the confidentiality operation modes and does not need any parameter. You can consult the library documentation for this mode here: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.modes.ECB

### Task 1.3.1 Encrypt/decrypt with AES 256 ECB PKCS7

#### TASK

<mark>Write a new encryption and decryption code based on your AES256-CBC-PKCS7 code but that makes use of the ECB mode instead of the CBC one. Wrap the code in two functions (headers given below).</mark>

In [19]:
def aes256_ecb_pkcs7_encrypt(data_to_encrypt, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    padder = padding.PKCS7(128).padder()

    padded_message = padder.update(data_to_encrypt) + padder.finalize()

    encryptor = cipher.encryptor()
    ct = encryptor.update(padded_message) + encryptor.finalize()
    
    return ct

def aes256_ecb_pkcs7_decrypt(encrypted_data, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    unpadder = padding.PKCS7(128).unpadder()
    decryptor = cipher.decryptor()
    padded_pt = decryptor.update(encrypted_data) + decryptor.finalize()
    unpadded_pt = unpadder.update(padded_pt) + unpadder.finalize()
    
    return unpadded_pt

#### CHECK

You should obtain the following results if you encrypt <code>message</code> with the same <code>key</code> previously used.
<code>
message = b'a secret message'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
padded_message = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10' 
length =  32
ciphertext = b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F|\x98\xfa\x0b\x8e\xab\xd4\xd7\x8e\x8f\xf5\n!&\xfd\xbb'
hex_ciphertext = 69b0611b958f16d6ed31c6fca5abd5467c98fa0b8eabd4d78e8ff50a2126fdbb
padded_plaintext = b'a secret message\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
hex_padded_plaintext = 6120736563726574206d65737361676510101010101010101010101010101010
plaintext = b'a secret message'
hex_plaintext = 6120736563726574206d657373616765
</code>

In [20]:
message = b'a secret message'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'

ciphertext = aes256_ecb_pkcs7_encrypt(message, key)
print("ciphertext = ", ciphertext)

plaintext = aes256_ecb_pkcs7_decrypt(ciphertext, key)
print("plaintext = ", plaintext)

ciphertext =  b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F|\x98\xfa\x0b\x8e\xab\xd4\xd7\x8e\x8f\xf5\n!&\xfd\xbb'
plaintext =  b'a secret message'


### Task 1.3.2 Checking AES 256 ECB PKCS7 

#### TASK

<mark>Test your code now with the message <code>other_message</code>, with a length in char not multiple of 16. </mark>

In [22]:
other_message = b"This message is important"
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'

ciphertext = aes256_ecb_pkcs7_encrypt(other_message, key)
print("ciphertext = ", ciphertext)

plaintext = aes256_ecb_pkcs7_decrypt(ciphertext, key)
print("plaintext = ", plaintext)

ciphertext =  b'\xb8{\xe1\x8e\x9fc\xfb\xaa\x13y\xb6\xf5VQW\xc4\xa1\xd0\xc1! .\xff\x9a\xb4\xc9X\x01\xc2\xc9\xf9\xd0'
plaintext =  b'This message is important'


#### CHECK

You should obtain the following results:
<code>
other_message = b'This message is important'
padded_other_message = b'This message is important\x07\x07\x07\x07\x07\x07\x07' 
length =  32
ciphertext = b'\xb8{\xe1\x8e\x9fc\xfb\xaa\x13y\xb6\xf5VQW\xc4\xa1\xd0\xc1! .\xff\x9a\xb4\xc9X\x01\xc2\xc9\xf9\xd0'
hex_ciphertext = b87be18e9f63fbaa1379b6f5565157c4a1d0c121202eff9ab4c95801c2c9f9d0
padded_plaintext = b'This message is important\x07\x07\x07\x07\x07\x07\x07'
hex_padded_plaintext = 54686973206d65737361676520697320696d706f7274616e7407070707070707
plaintext = b'This message is important'
hex_plaintext = 54686973206d65737361676520697320696d706f7274616e74
</code>

### Task 1.3.3 Checking AES 256 ECB PKCS7 against CyberChef

#### TASK

<mark>Test again your AES256-ECB-PKCS7 function but this time check your results using the web page tool CyberChef: https://gchq.github.io/CyberChef/ . You have to select, in the left sidebar, 'AES Encrypt' from 'Encryption / Encoding' and ECB mode. </mark>

### Task 1.3.4 Generating cryptographically secure random bytes

Till now you have used fixed and given key and iv in your AES256 code. However, ideally, you should randomly generate these parameters each time you need them, at least in the first encryption step. Notice that it is not the same case in the decryption step, as here you must use the same parameters used in the encryption step if you want to correctly recover the plaintext.

Let’s learn first how to generate <ins>cryptographically secure</ins> random bytes.

Read the information provided in the web page section: https://cryptography.io/en/latest/random-numbers/#random-number-generation

Two possibilities are highlighted:

- Using the function <code>urandom(..)</code> from the os module (https://docs.python.org/3/library/os.html#os.urandom)

- Using the function <code>token_bytes(..)</code> from the secrets module (https://docs.python.org/3/library/secrets.html#secrets.token_bytes)

You should have imported both in Section 0.

#### TASK

<mark>Try both ways with the code shown next. Notice that each time you execute the code, new bytes are obtained.</mark>

In [24]:
# import os 
# import secrets

print(os.urandom(16))            # generating 16 random bytes
print(secrets.token_bytes(32))   # generating 32 random bytes

b'i\x905\x14\xa4&BK-\xbd;\x13\xd6\x10\xa0\x90'
b's\x8f>t\x8e\x07\r\xe4o\x17\xd3\xbb7AJ\x17Z\x8b%\x86-\xa5\x9a)\xb8\x8bWV\x19\xec\xd6\xff'


### Task 1.3.5 No-key AES 256 ECB PKCS7 and No-key_no-iv AES 256 CBC PKCS7 encryption functions

Let’s change the your AES256-ECB-PKCS7 encryption code so if no key is provided, the key is randomly generated. Both the ciphertext and the key should be provided as output. Use the code provided below 14 as guide to write a new function <code>aes256_ecb_pkcs7_encrypt_rnd(..)</code> that incorporates this functionality. 

Note that the function expects two parameters but by writing <code>key=None</code> in the function header for the second one, we allow its absence (that is, it is optional to call the function with a second parameter) and at the same time we provide a default value if it is not provided (in this case this default value is the <code>None</code> value). This means also that, if the key value is not provided, we have to provide a key value inside the function code (in the provided code <code>os.urandom(..)</code> is used to generate this value randomly). As the key value is needed for decryption, notice that we also return it together with the resulting ciphertext. 

The same happens with the initialization vector <code>iv</code> in the case of the CBC mode. Notice that in the case of AES cipher, the <code>iv</code> size is the same as the block size, that is, 16 bytes.

#### TASK

<mark>Re-write the encryption functions as explained taking the following headers and code as starting point.</mark>

In [25]:
def aes256_ecb_pkcs7_encrypt_rnd(data_to_encrypt, key=None):
    if key is None:
        key = os.urandom(32) 

    cipher = Cipher(algorithms.AES(key), modes.ECB())
    padder = padding.PKCS7(128).padder()

    padded_message = padder.update(data_to_encrypt) + padder.finalize()

    encryptor = cipher.encryptor()
    ct = encryptor.update(padded_message) + encryptor.finalize()
    
    return ct, key

def aes256_cbc_pkcs7_encrypt_rnd(data_to_encrypt, key=None, iv=None):
    if not key:
        key = os.urandom(16)
    
    if not iv:
        iv = os.urandom(16)
    
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    padder = padding.PKCS7(128).padder()

    padded_message = padder.update(data_to_encrypt) + padder.finalize()

    encryptor = cipher.encryptor()
    ct = encryptor.update(padded_message) + encryptor.finalize()
    
    return ct, key, iv

#### CHECK

Once you have the code running, try to encrypt and decrypt the same message and check that different keys and ciphertext are obtained.

In [26]:
# CHECK ECB FUNCTION

print("# ECB FIRST RUN")
ciphertext1, randomkey1 = aes256_ecb_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey1 =", randomkey1.hex(), "\nhex_ciphertext1 =", ciphertext1.hex())
plaintext1 = aes256_ecb_pkcs7_decrypt(ciphertext1, randomkey1)
print("plaintext1 = ", plaintext1)

print("\n# ECB SECOND RUN")
ciphertext2, randomkey2 = aes256_ecb_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey2 =", randomkey2.hex(), "\nhex_ciphertext2 =", ciphertext2.hex())
plaintext2 = aes256_ecb_pkcs7_decrypt(ciphertext2, randomkey2)
print("plaintext2 = ", plaintext2)

# ECB FIRST RUN
hex_randomkey1 = b8f5d67ba237807419047df64f08bc6881883f61b459a0c79c5f0544278ef523 
hex_ciphertext1 = 83a29cb592bf103a21d2654c2d6b20142434d6912a02f0a2a780c99fe7b3fd73
plaintext1 =  b'This message is important'

# ECB SECOND RUN
hex_randomkey2 = ac00235a3a55f6ed6cbffa44eafe53b5fc8021a7d0950022c9dab91282a2f148 
hex_ciphertext2 = 7b788b6b9d79f66e372e4109abadeaaab0e9b8f221b28922d154d838cf3a3730
plaintext2 =  b'This message is important'


In [29]:
# CHECK CBC FUNCTION

print("# CBC FIRST RUN")
ciphertext1, randomkey1, randomiv1= aes256_cbc_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey1 =", randomkey1.hex(), "\nhex_ciphertext1 =", ciphertext1.hex(), "\nhex_randomiv1 =", randomiv1.hex())
plaintext1 = aes256_cbc_pkcs7_decrypt(ciphertext1, randomkey1, randomiv1)
print("plaintext1 = ", plaintext1)

print("\n# CBC SECOND RUN")
ciphertext2, randomkey2, randomiv2 = aes256_cbc_pkcs7_encrypt_rnd(b"This message is important")
print("hex_randomkey2 =", randomkey2.hex(), "\nhex_ciphertext2 =", ciphertext2.hex(), "\nhex_randomiv2 =", randomiv2.hex())
plaintext2 = aes256_cbc_pkcs7_decrypt(ciphertext2, randomkey2, randomiv2)
print("plaintext2 = ", plaintext2)

# CBC FIRST RUN
hex_randomkey1 = a37dccad9a3a9a35fbc97874b57b7384 
hex_ciphertext1 = a1fabb51ff92f22ceb8763abc91c01c77580f8d57609925b35d36f8128459c12 
hex_randomiv1 = 7c50573b04102d2ffb82f568f33ffaff
plaintext1 =  b'This message is important'

# CBC SECOND RUN
hex_randomkey2 = d54d4a3225f3fb203e048a566bc8bffd 
hex_ciphertext2 = ea5a0855ba9b796354b7069bb1b02bb4124aff7ec5e0b7fdefeb0e6034e82c3c 
hex_randomiv2 = e4314e095f21d0230fab66db4a7cb66f
plaintext2 =  b'This message is important'


## 1.4 AES256 - CTR MODE

Let’s now use the CTR operation mode. You can consult the library documentation for this mode here: 
https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.modes.CTR 
Notice that in this case you do not need an initialization vector, but you need a nonce to initialize the counter. 

### Task 1.4.1 Encrypt/decrypt with AES 256 CTR

#### QUESTION

<mark>Review your notes on CTR operation mode. Does CTR operation mode need padding? That is, is a requirement that the length of the data to encrypt in bytes is multiple of the cipher block size?</mark>

No, CTR operation mode does not need padding.

#### TASK

<mark>Write two new encryption and one decryption functions based on your AES256-CBC code but that makes use of the CTR mode instead of the CBC one. Notice that PKCS7 padding will NOT be applied.</mark>

In [36]:
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'

def aes256_ctr_encrypt(data_to_encrypt, key, nonce):
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce))

    encryptor = cipher.encryptor()
    ct = encryptor.update(data_to_encrypt) + encryptor.finalize()
    
    return ct 


def aes256_ctr_decrypt(encrypted_data, key, nonce):
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce))
    decryptor = cipher.decryptor()
    padded_pt = decryptor.update(encrypted_data) + decryptor.finalize()
    
    return padded_pt 


def aes256_ctr_encrypt_rnd(data_to_encrypt, key=None, nonce=None):
    if not key:
        key = os.urandom(16)
    
    if not nonce:
        nonce = os.urandom(16)

    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce))

    encryptor = cipher.encryptor()
    ct = encryptor.update(data_to_encrypt) + encryptor.finalize()
    
    return ct 

#### CHECK

In [37]:
# CHECK YOUR CODE

nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'

print("other_message :", other_message)
print("key =", key)
print("hex_key =", key.hex())
print("nonce =", nonce)
print("hex_nonce =", nonce.hex())

ciphertext1 = aes256_ctr_encrypt(other_message, key, nonce)
ciphertext2 = aes256_ctr_encrypt(other_message+other_message, key, nonce)

print("ciphertext1 =", ciphertext1.hex(), "\nlength ciphertext1 =", len(ciphertext1))
print("ciphertext2 =", ciphertext2.hex(), "\nlength ciphertext2 =", len(ciphertext2))

plaintext1 = aes256_ctr_decrypt(ciphertext1, key, nonce)
plaintext2 = aes256_ctr_decrypt(ciphertext2, key, nonce)

print("plaintext1 =", plaintext1)
print("plaintext2 =", plaintext2)

other_message : b'This message is important'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'
hex_nonce = f42489ab7ba42d603ade71b56d1bd398
ciphertext1 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8 
length ciphertext1 = 25
ciphertext2 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8e6e3ceba38e8690d5be581bbd93f69a7236edf40d9d900ae96 
length ciphertext2 = 50
plaintext1 = b'This message is important'
plaintext2 = b'This message is importantThis message is important'


You should obtain the following results if you encrypt <code>other_message</code> and two times <code>other_message</code> with the same <code>key</code> previously used and the provided <code>nonce</code>.
<code>
other_message : b'This message is important'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'
hex_nonce = f42489ab7ba42d603ade71b56d1bd398
ciphertext1 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8 
length ciphertext1 = 25
ciphertext2 = a79a1e04ca64996bb6a2644692ec77449d189e30177d2992d8e6e3ceba38e8690d5be581bbd93f69a7236edf40d9d900ae96 
length  ciphertext2 = 50
plaintext1 = b'This message is important'
plaintext2 = b'This message is importantThis message is important'
</code>

#### QUESTION

<mark>Which length in bytes has the input data to encrypt in Task 1.4.1? And the output ciphertext? Is the length of the output ciphertext multiple of the cipher block size or not and why?</mark> 

In [38]:
print("Length of input data: ", len(other_message))

Length of input data:  25


The length of the input data is 25 bytes. The ciphertext is also a length of 25 bytes, which is not a multiple of the cipher block size. This is because CTR mode effectively does stream encryption, so it will encrypt every single byte.

### Task 1.4.2 Checking AES 256 CTR against CyberChef

#### TASK

<mark>Test again your AES256-CTR code but this time check your results using the web page tool CyberChef: https://gchq.github.io/CyberChef/ </mark>

## 1.5 ADDITIONAL ASPECTS OF OPERATION MODES

### Task 1.5.1 No-padding AES 256 ECB and CBC functions

You have already been questioned about the difference in the output lengths when different operation modes are used and when padding is added or not. Now you’ll study what happens when you intentionally modify one bit (or octet) of a ciphertext and then decrypt the modified ciphertext. 

In order to play with modifications in the ciphertext, it is better to avoid PKCS7 padding, as if modifications affect the padding, it will probably raise an error and you’ll not be able to study the effects of the modifications in the plaintext after decryption. 

#### TASK 

<mark>Write a new set of functions for AES256-CBC and AES256-ECB encryption and decryption that do not apply padding.</mark>

In [40]:
def aes256_cbc_encrypt(data_to_encrypt, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))

    encryptor = cipher.encryptor()
    ct = encryptor.update(data_to_encrypt) + encryptor.finalize()
    
    return ct
    
def aes256_cbc_decrypt(encrypted_data, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(encrypted_data) + decryptor.finalize()
    
    return plaintext

def aes256_ecb_encrypt(data_to_encrypt, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB())

    encryptor = cipher.encryptor()
    ct = encryptor.update(data_to_encrypt) + encryptor.finalize()
    
    return ct

def aes256_ecb_decrypt(encrypted_data, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(encrypted_data) + decryptor.finalize()
    
    return plaintext


#### CHECK

Test your functions by encrypting messages with length in chars multiple of 16 and then decrypting the resulting ciphertext.

In [41]:
# CHECKING YOUR CODE

print("message :", message)
print("key :", key)
print("hex_key :", key.hex())
print("iv :", iv)
print("hex_iv :", iv.hex())

print("AES256-CBC ciphertext: ", aes256_cbc_encrypt(message, key, iv))
print("AES256-CBC plaintext: ", aes256_cbc_decrypt(aes256_cbc_encrypt(message, key, iv), key, iv))

print("AES256-ECB ciphertext: ", aes256_ecb_encrypt(message, key))
print("AES256-ECB plaintext: ", aes256_ecb_decrypt(aes256_ecb_encrypt(message, key), key))

message : b'a secret message'
key : b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key : 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
iv : b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
hex_iv : 7fb357841a394b4e7eaf73fc9fb6ccb8
AES256-CBC ciphertext:  b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
AES256-CBC plaintext:  b'a secret message'
AES256-ECB ciphertext:  b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F'
AES256-ECB plaintext:  b'a secret message'


With the previously used values for <code>message</code>, <code>key</code> and <code>iv</code>, you should obtain the following results:
<code>
message : b'a secret message'
key : b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key : 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
iv : b'\x7f\xb3W\x84\x1a9KN~\xafs\xfc\x9f\xb6\xcc\xb8'
hex_iv : 7fb357841a394b4e7eaf73fc9fb6ccb8
AES256-CBC ciphertext:  b';\xa6\xe5\x85;^\xc2&\xec\x07|\x03\xcc\r\xaeN'
AES256-CBC plaintext:  b'a secret message'
AES256-ECB ciphertext:  b'i\xb0a\x1b\x95\x8f\x16\xd6\xed1\xc6\xfc\xa5\xab\xd5F'
AES256-ECB plaintext:  b'a secret message'
</code>

### Task 1.5.2 Effects of modifying ciphertext under different operation modes 

#### TASK

<mark>Now, analyse how the decrypted plaintext changes after modifying one bit of the ciphertext at different positions (one in the first block, another in the second block…).</mark>

Suggested steps:
- Select a message with at least 48-char length (remember that the length in char has to be multiple of 16)
- Select a key, and iv and nonce if necessary
- Encrypt the message operating the cipher with the selected mode
- Save the ciphertext in one variable (ciphertext)
- Modify the previous ciphertext in one bit and save the result in another variable (modified_ciphertext)
- Decrypt both the ciphertext and its modified version, saving the decryptions, respectively, in two variables plaintext and modified_plaintext
- Analyse how the ciphertext modification has affected the plaintext

In [56]:
message = b'This is a 48-character long sample message! 1234'
modified_byte_mask = 0x01

# CBC
cbc_ciphertext = aes256_cbc_encrypt(message, key, iv)

# Modifying one byte in ciphertext
cbc_modified_ciphertext = cbc_ciphertext[:len(cbc_ciphertext)-1] + bytes([cbc_ciphertext[-1] ^ 1])

# Decrypting CBC
cbc_decrypted = aes256_cbc_decrypt(cbc_ciphertext, key, iv)
cbc_modified_decrypted = aes256_cbc_decrypt(cbc_modified_ciphertext, key, iv) 
print("Decrypted ciphertext via CBC: ", cbc_decrypted)
print("Decrypted modified ciphertext via CBC: ", cbc_modified_decrypted)


# ECB
ecb_ciphertext = aes256_ecb_encrypt(message, key)

# Modifying one byte in ciphertext
ecb_modified_ciphertext = ecb_ciphertext[:len(ecb_ciphertext)-1] + bytes([ecb_ciphertext[-1] ^ 1])

# Decrypting CBC
ecb_decrypted = aes256_ecb_decrypt(ecb_ciphertext, key)
ecb_modified_decrypted = aes256_ecb_decrypt(ecb_modified_ciphertext, key) 
print("Decrypted ciphertext via ECB: ", ecb_decrypted)
print("Decrypted modified ciphertext via ECB: ", ecb_modified_decrypted)


# CTR
ctr_ciphertext = aes256_ctr_encrypt(message, key, nonce)

# Modifying one byte in ciphertext
ctr_modified_ciphertext = ctr_ciphertext[:len(ctr_ciphertext)-1] + bytes([ctr_ciphertext[-1] ^ 1])

# Decrypting CBC
ctr_decrypted = aes256_ctr_decrypt(ctr_ciphertext, key, nonce)
ctr_modified_decrypted = aes256_ctr_decrypt(ctr_modified_ciphertext, key, nonce) 
print("Decrypted ciphertext via CTR: ", ctr_decrypted)
print("Decrypted modified ciphertext via CTR: ", ctr_modified_decrypted)


Decrypted ciphertext via CBC:  b'This is a 48-character long sample message! 1234'
Decrypted modified ciphertext via CBC:  b"This is a 48-character long samp'\xf8a\x8a\x91\xfa/\x04\xf7\xbe\xd11\xed\x02\\\xfa"
Decrypted ciphertext via ECB:  b'This is a 48-character long sample message! 1234'
Decrypted modified ciphertext via ECB:  b'This is a 48-character long samp\x0b\xd2\xa3\\."\xa7\xb8S\x8a;\xa9\x15\xa4M\x8d'
Decrypted ciphertext via CTR:  b'This is a 48-character long sample message! 1234'
Decrypted modified ciphertext via CTR:  b'This is a 48-character long sample message! 1235'


#### QUESTION

<mark>How many bits (or bytes) are affected in each case (ECB, CBC, CTR)? Which are their positions? Can you explain the reasons in each case considering the course notes on operation modes?</mark>

ECB: the block with the affected bit will be affected. This is because ECB encrypts and decrypts each block independently.

CBC: the block with the affected bit and the following block will be affected. This is because CBC XORs the previous ciphertext block with the next plaintext, which will mangle the bits.

CTR: only one bit will be affected. This is because CTR is effectively stream encrypting.

## 1.6 OTHER CIPHERS AND KEY SIZES

### Task 1.6.1 Camellia symmetric block cipher

#### TASK

<mark>Now, create new functions similar to the ones already coded but that in this case use the Camellia cipher and other key sizes. Consult the library documentation to know more about the Camellia cipher and available key sizes for both AES and Camellia.</mark>

In [61]:
def camellia_ecb_encrypt(data_to_encrypt, key):
    cipher = Cipher(algorithms.Camellia(key), modes.ECB())

    encryptor = cipher.encryptor()
    ct = encryptor.update(data_to_encrypt) + encryptor.finalize()
    
    return ct
    
def camellia_ecb_decrypt(encrypted_data, key):
    cipher = Cipher(algorithms.Camellia(key), modes.ECB())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(encrypted_data) + decryptor.finalize()
    
    return plaintext

In [63]:
print("message = ", message)

encrypted_camellia_msg = camellia_ecb_encrypt(message, key)
print("encrypted camellia msg: ", encrypted_camellia_msg)

decrypted_camellia_msg = camellia_ecb_decrypt(encrypted_camellia_msg, key)
print("decrypted camellia msg: ", decrypted_camellia_msg)

message =  b'This is a 48-character long sample message! 1234'
encrypted camellia msg:  b'[\xbdd\xa9\x7f\xcb~S\xc4\xd5N\xf0}\xbe\xc6<A\xe7\xd0P\xd7\xb6g\tm\xdd\x01\x99\xc3\xf1\xc6C\xff\xdb\xcd~\xffj]\x03P\x08<\n\xa2\xc45\xa8'
decrypted camellia msg:  b'This is a 48-character long sample message! 1234'


## 1.7 EXCHANGING SECRET MESSAGES WITH A PARTNER

### Task 1.7.1 Exchanging secret messages

#### TASK

<mark>Choose a partner among your classmates and exchange with him/her secret messages. You’ll need to send the key and other needed parameters along with the ciphertext. The receiver will also need to know the used algorithm, operation mode and key/block size if there are available more than one.</mark>

**Notice that in real life keys should NEVER be sent UNPROTECTED.**

# 2. Symmetric Stream Ciphers

Next, you’ll learn how to encrypt a simple text with symmetric stream ciphers. First, you’ll use the CHACHA20 cipher and then you’ll use the pylfsr library to generate pseudorandom key streams using LFSRs.

## 2.1 CHACHA20 STREAM CIPHER

### Task 2.1.1 CHACHA20 encryption/decryption functions

#### TASK

Now read the information and take the code provided as example in the following web page section: 
https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.ChaCha20

Take the code in the example and use it as base to create the functions whose headers are defined next. Test your functions with the data and results of Code Snippet 29 and Code Snippet 30. 

In [67]:
import struct

def chacha20_encrypt(message, key, nonce):
    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None)

    encryptor = cipher.encryptor()
    ct = encryptor.update(message) + encryptor.finalize()
    
    return ct


def chacha20_encrypt_rnd(message, key=None, nonce=None):
    if key is None:
        key = os.urandom(32)    # key must have 256 bits
    if nonce is None:
       nonce = os.urandom(16)   # nonce must have 128 bits
    
    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None)

    encryptor = cipher.encryptor()
    ct = encryptor.update(message) + encryptor.finalize()
    
    return ct, key, nonce
    

def chacha20_decrypt(ciphertext, key, nonce):
    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None)
    
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    
    return plaintext

#### CHECK

In [68]:
# Check your code

print("message :", message)
print("other_message :", other_message)
print("key =", key)
print("hex_key =", key.hex())
print("nonce =", nonce)
print("hex_nonce =", nonce.hex())

print("ciphertext chacha20 message=", chacha20_encrypt(message, key, nonce))
print("hex_ciphertext chacha20 message=", chacha20_encrypt(message, key, nonce).hex())

print("plaintext chacha20 message=", chacha20_decrypt(chacha20_encrypt(message, key, nonce), key, nonce))
print("hex_plaintext chacha20 message=", chacha20_decrypt(chacha20_encrypt(message, key, nonce), key, nonce).hex())

print("ciphertext chacha20 other_message=", chacha20_encrypt(other_message, key, nonce))
print("hex_ciphertext chacha20 other_message=", chacha20_encrypt(other_message, key, nonce).hex())

print("plaintext chacha20 other_message=", chacha20_decrypt(chacha20_encrypt(other_message, key, nonce), key, nonce))
print("hex_plaintext chacha20 other_message=", chacha20_decrypt(chacha20_encrypt(other_message, key, nonce), key, nonce).hex())


message : b'This is a 48-character long sample message! 1234'
other_message : b'This message is important'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'
hex_nonce = f42489ab7ba42d603ade71b56d1bd398
ciphertext chacha20 message= b'dJ\x8c\x84\xae\xda\x86y\xd6?\xfbdp\xb1o\xf9\x02K\xb4:\xbb1\xac:a\xecD\xae\xceHi\xa0\xd0\xc0f\xd6\xa2n\x00X\x06\xec<M\x0b\xff\xf0f'
hex_ciphertext chacha20 message= 644a8c84aeda8679d63ffb6470b16ff9024bb43abb31ac3a61ec44aece4869a0d0c066d6a26e005806ec3c4d0bfff066
plaintext chacha20 message= b'This is a 48-character long sample message! 1234'
hex_plaintext chacha20 message= 5468697320697320612034382d636861726163746572206c6f6e672073616d706c65206d657373616765212031323334
ciphertext chacha20 other_message= b'dJ\x8c\x84\xae\xde\x90*\xc4~\xa89}\xbbt\xb8\x19G\xa7!\xac7\xed8z'
hex_ciphertex

You should obtain the following result:
<code>
message : b'a secret message'
other_message : b'This message is important'
key = b'MN\\\xde\xa1\xf7\x9c\xcf\x10A\xa8\x1e\x00\x0eeq\xe86\x1c\x12\x18\xa6\x10\xfb\xb9^\n,0\xa2kj'
hex_key = 4d4e5cdea1f79ccf1041a81e000e6571e8361c1218a610fbb95e0a2c30a26b6a
nonce = b'\xf4$\x89\xab{\xa4-`:\xdeq\xb5m\x1b\xd3\x98'</code>

<code>hex_nonce = f42489ab7ba42d603ade71b56d1bd398
ciphertext chacha20 message= b'Q\x02\x96\x92\xed\xc1\x90-\x97r\xaa/.\xb3`\xfd'
hex_ciphertext chacha20 message= 51029692edc1902d9772aa2f2eb360fd
plaintext chacha20 message= b'a secret message'
hex_plaintext chacha20 message= 6120736563726574206d657373616765
ciphertext chacha20 other_message= b'dJ\x8c\x84\xae\xde\x90*\xc4~\xa89}\xbbt\xb8\x19G\xa7!\xac7\xed8z'
hex_ciphertext chacha20 other_message= 644a8c84aede902ac47ea8397dbb74b81947a721ac37ed387a
plaintext chacha20 other_message= b'This message is important'
hex_plaintext chacha20 other_message= 54686973206d65737361676520697320696d706f7274616e74  
</code>